In [1]:
%matplotlib inline

# Extract data from mongoDB and generate geometries.

Created on:  2016-10-27  
Last update: 2016-11-10  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

In [74]:
# preliminaries
from __future__ import unicode_literals
import sys
import csv
import os
import math
import pprint
import requests
import gzip
from collections import defaultdict
import time
import datetime
import numpy as np
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry

import json
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from functools import partial
import pyproj
from scipy import spatial
from haversine import haversine
import overpass
apiop = overpass.API(timeout=600)

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt
# import mpld3
# mpld3.enable_notebook() # unfortunately, this is too buggy: https://github.com/mpld3/mpld3/issues/193
# mpld3.disable_notebook()

pathdatain = '/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/'
pathdataout = '/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/derived/'
pathdataoutfinal = '/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/scripts/mobviz/d3unroll/'
curtime = time.strftime("%Y%m%d_%H%M%S")
logging.basicConfig(filename=pathdataout+'mobviz_streetalgorithm'+curtime+'.log',level=logging.DEBUG)
pp = pprint.PrettyPrinter(indent=4)

# City parameters (generalize later)
cityname = "vienna"
bbox = "52.33812, 13.0884, 52.675499, 13.76134" 
# berlin: 52.33812, 13.0884, 52.675499, 13.76134
# newyork: 40.496044, -74.255735, 40.915256, -73.700272
# stuttgart: 48.46, 8.79, 48.93, 9.50
# amsterdam: 52.3216, 4.7685, 52.4251, 5.0173
postalcodes = []
with open(pathdatain+"berlin/"+"berlin_postalcodes.txt") as f:
    for line in f:
        postalcodes.append(int(line))

In [75]:
# mongo connection
client = MongoClient()
dbin = client[cityname+'_raw']
nodesin = dbin['nodes']
cursor = nodesin.find({})
numnodes = cursor.count()
waysin = dbin['ways']
cursor = waysin.find({})
numways = cursor.count()

dbout = client[cityname+'_derived']
dbout.drop_collection('ways')
dbout = client[cityname+'_derived']
waysout = dbout['ways']

In [76]:
bar = pyprind.ProgBar(numways, bar_char='█', update_interval=1)
for i,way in enumerate(waysin.find({})):
    bar.update(item_id = i)
#     pp.pprint(way)
    tempgeojson = {}
    tempgeojson["_id"] = way["_id"]
    try:
        tempgeojson["properties"] = way['tags']
    except:
        tempgeojson["properties"] = {}
    tempgeojson["type"] = "Feature"
    tempgeojson["geometry"] = {"type":"", "coordinates":[]} 
    coords = []
    for nodeid in way["loc"]["nodes"]:
        for n in nodesin.find({"_id": nodeid}):
            coords.append([n["loc"]["coordinates"][0], n["loc"]["coordinates"][1]])
    tempgeojson["geometry"]["coordinates"] = coords
    if way["loc"]["nodes"][0] == way["loc"]["nodes"][-1]:
        tempgeojson["geometry"]["type"] = "Polygon"
    else:
        tempgeojson["geometry"]["type"] = "LineString"
#     pp.pprint(tempgeojson)
#     waysout.update_one({'_id':way["_id"]}, tempgeojson, upsert=True)
    waysout.insert_one(tempgeojson)

0%                          100%
[█████████████████████████████ ] | ETA: 00:00:00 | Item ID: 342859
Total time elapsed: 00:12:50


## Calculate lengths and areas of ways and save back to mongoDB

In [77]:
client = MongoClient()
db = client[cityname+'_derived']
ways = db['ways']

In [78]:
cursor = ways.find({"geometry.type": "LineString"})
numLineStrings = cursor.count()
bar = pyprind.ProgBar(numLineStrings, bar_char='█', update_interval=1)
for i,way in enumerate(ways.find({"geometry.type": "LineString"})):
    bar.update(item_id = i)
    npway = np.asarray(way["geometry"]["coordinates"])
    distances = [1000*haversine(npway[i][::-1], npway[i+1][::-1]) for i in range(npway.shape[0]-1)]
    ways.update_one({'_id': way["_id"]}, {"$set": {"properties_derived.length": sum(distances)}}, upsert=False)

cursor = ways.find({"geometry.type": "Polygon"})
numPolygons = cursor.count()
bar = pyprind.ProgBar(numPolygons, bar_char='█', update_interval=1)
for i,way in enumerate(ways.find({"geometry.type": "Polygon"})):
    bar.update(item_id = i)
    npway = np.asarray(way["geometry"]["coordinates"])
    distances = [1000*haversine(npway[i][::-1], npway[i+1][::-1]) for i in range(npway.shape[0]-1)]
    ways.update_one({'_id': way["_id"]}, {"$set": {"properties_derived.length": sum(distances)}}, upsert=False)
    # Following area calculating code from: http://gis.stackexchange.com/questions/127607/area-in-km-from-polygon-of-coordinates
    try: # IllegalArgumentException: Invalid number of points in LinearRing found 3 - must be 0 or >= 4
        geom = Polygon(npway)
        geom_area = ops.transform(
            partial(
                pyproj.transform,
                pyproj.Proj(init='EPSG:4326'),
                pyproj.Proj(
                    proj='aea',
                    lat1=geom.bounds[1],
                    lat2=geom.bounds[3])),
            geom)
        # Export the area in m^2
        ways.update_one({'_id': way["_id"]}, {"$set": {"properties_derived.area": geom_area.area}}, upsert=False)
    except:
        print("Something went wrong: " + str(i))
        pass

0%                          100%
[█████████████████████████████ ] | ETA: 00:00:00 | Item ID: 86434
Total time elapsed: 00:00:32
0%                          100%
[                              ]IllegalArgumentException: Invalid number of points in LinearRing found 3 - must be 0 or >= 4
Shell is not a LinearRing


Something went wrong: 135
Something went wrong: 320


[                              ] | ETA: 00:08:30 | Item ID: 498IllegalArgumentException: Invalid number of points in LinearRing found 3 - must be 0 or >= 4
Shell is not a LinearRing


Something went wrong: 470
Something went wrong: 555
Something went wrong: 659
Something went wrong: 1050


[                              ] | ETA: 00:06:48 | Item ID: 1243IllegalArgumentException: Invalid number of points in LinearRing found 3 - must be 0 or >= 4
Shell is not a LinearRing


...

[█████████████████████████████ ] | ETA: 00:00:01 | Item ID: 253913

Something went wrong: 254198


[█████████████████████████████ ] | ETA: 00:00:00 | Item ID: 254662

Something went wrong: 254870



Total time elapsed: 00:05:32
